In [7]:
import pandas as pd


In [56]:
data= pd.read_csv('./spotify_millsongdata.csv')


In [57]:
data.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [58]:
data.shape

(57650, 4)

In [59]:
data.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [60]:
data= data.drop('link', axis=1).reset_index(drop=True)
data.head(10)


,artist,song,text
0,ABBA,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante","Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,Making somebody happy is a question of give an...
5,ABBA,Burning My Bridges,"Well, you hoot and you holler and you make me ..."
6,ABBA,Cassandra,Down in the street they're all singing and sho...
7,ABBA,Chiquitita,"Chiquitita, tell me what's wrong \r\nYou're e..."
8,ABBA,Crazy World,I was out with the morning sun \r\nCouldn't s...
9,ABBA,Crying Over You,I'm waitin' for you baby \r\nI'm sitting all ...


In [ ]:
data=data.sample(20000).reset_index(drop=True)
data.shape

(20000, 3)

In [52]:
data.head(10)

,artist,song,text
0,Radiohead,Killer Cars,too hard on the brake again what if these brak...
1,Isley Brothers,You're All I Need,"yeah , yeah , yeah , yeah , yeah , all alon fo..."
2,Kelly Clarkson,Up To The Mountains,i went up to the mountain becaus you ask me to...
3,Chaka Khan,Earth To Mickey,come along let 's park ( let 's park ) ca n't ...
4,Whiskeytown,Pawn Shop Ain't No Place For A Wedding Ring,a pawn shop ai n't no place for a wed ring six...
5,Wham!,Bad Boys,"woo-woo dear mummi , dear daddi , you had plan..."
6,Irving Berlin,Slumming On Park Avenue,put on your slum cloth and get your car let 's...
7,Sting,Desert Rose,i dream of rain i dream of garden in the deser...
8,Dewa 19,Kosong,[ vers 1 ] kamu seperti hantu teru menghantuik...
9,Matt Monro,When Sunny Gets Blue,when sunni get blue sky get grey and cloudi an...


 Data Preprocessing


In [64]:
data['text']= data['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n\r', ' ', regex=True)


In [28]:
import nltk
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
stemmer= PorterStemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/priteshdube/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [16]:
def tokenization(txt):
  tokens= nltk.word_tokenize(txt)
  stemming= [stemmer.stem(w) for w in tokens]
  return " ".join(stemming)


In [29]:
data['text']= data['text'].apply(lambda x: tokenization(x))

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [31]:
Tfid= TfidfVectorizer(analyzer='word', stop_words='english')
matrix= Tfid.fit_transform(data['text'])
similarity= cosine_similarity(matrix)

In [46]:
similarity[9]

array([0.        , 0.01971345, 0.0766589 , ..., 0.01554872, 0.01567745,
       0.00238071])

In [67]:
data[data['song']=='Cassandra'].index[0]

6

Recommender

In [41]:
def recommend(song):
  idx= data[data['song']==song].index[0]
  distance= sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x:x[1])
  song_list=[]
  for s in distance[1:11]:
    song_list.append(data.iloc[s[0]].song)
  return song_list

In [49]:
recommend('Cassandra')

['Sorry',
 "Don't Tell Me You're Sorry",
 'Sorry Is A Sorry Word',
 'Sorry',
 'All Hail Me',
 "After I Say I'm Sorry?",
 "I'm Sorry",
 'A Woman Crossed In Love',
 'Hearts On Fire',
 'Target Audience']

In [25]:
import pickle


In [26]:
pickle.dump(similarity, open('similarity.pkl', 'wb'))


In [27]:
pickle.dump(data, open('data.pkl', 'wb'))